In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from datetime import datetime, date
plt.style.use('ggplot')

In [2]:
trans = pd.read_csv('Transactions_Cleaned.csv')
cust = pd.read_csv('CustomerDemographic_Cleaned.csv')

In [9]:
trans.head(2)

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date,Profit
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,1970-01-01 00:00:00.000041245,17.87
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,1970-01-01 00:00:00.000041701,1702.55


In [10]:
cust.head(2)

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,Age
0,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,70
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0,43


In [11]:
print('Number of Record rows in Transcation = ', trans.shape[0])
print('Number of Record  columns in Transcation = ', trans.shape[1])
print()
print('Number of Record rows in customer demography = ', cust.shape[0])
print('Number of Record  columns in customer demography = ', cust.shape[1])

Number of Record rows in Transcation =  19803
Number of Record  columns in Transcation =  14

Number of Record rows in customer demography =  3912
Number of Record  columns in customer demography =  13


In [3]:
###  Let marge both table inner join
marge_trans_cust = pd.merge(trans, cust, left_on = 'customer_id', right_on = 'customer_id', how = 'inner')
marge_trans_cust.head(5)

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,...,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,Age
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,...,Male,19,1955-01-11,Software Engineer I,Financial Services,Mass Customer,N,Yes,10.0,68
1,11065,1,2950,2017-10-16,0.0,Approved,Giant Bicycles,Standard,medium,medium,...,Male,19,1955-01-11,Software Engineer I,Financial Services,Mass Customer,N,Yes,10.0,68
2,18923,62,2950,2017-04-26,0.0,Approved,Solex,Standard,medium,medium,...,Male,19,1955-01-11,Software Engineer I,Financial Services,Mass Customer,N,Yes,10.0,68
3,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,...,Female,89,1979-02-04,Clinical Specialist,Health,Mass Customer,N,Yes,10.0,44
4,6862,4,3120,2017-10-05,0.0,Approved,Giant Bicycles,Standard,high,medium,...,Female,89,1979-02-04,Clinical Specialist,Health,Mass Customer,N,Yes,10.0,44


In [13]:
marge_trans_cust.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19354 entries, 0 to 19353
Data columns (total 26 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   transaction_id                       19354 non-null  int64  
 1   product_id                           19354 non-null  int64  
 2   customer_id                          19354 non-null  int64  
 3   transaction_date                     19354 non-null  object 
 4   online_order                         19354 non-null  float64
 5   order_status                         19354 non-null  object 
 6   brand                                19354 non-null  object 
 7   product_line                         19354 non-null  object 
 8   product_class                        19354 non-null  object 
 9   product_size                         19354 non-null  object 
 10  list_price                           19354 non-null  float64
 11  standard_cost               

     After merge we found that Transcation date, DOB, and product_first_sold_date are not in date formate so 
    convert into date formet

In [14]:
marge_trans_cust['transaction_date'] = pd.to_datetime(marge_trans_cust['transaction_date'])
marge_trans_cust['DOB'] = pd.to_datetime(marge_trans_cust['DOB'])
marge_trans_cust['product_first_sold_date'] = pd.to_datetime(marge_trans_cust['product_first_sold_date'])

marge_trans_cust[['transaction_date', 'DOB', 'product_first_sold_date' ]].dtypes

transaction_date           datetime64[ns]
DOB                        datetime64[ns]
product_first_sold_date    datetime64[ns]
dtype: object

# RFM Analysis
RFM (Recency, Frequency, Monetary) analysis is a behavior-based approach grouping customers into segments. It groups the customers on the basis of their previous purchase transactions. How recently, how often, and how much did a customer buy. RFM filters customers into various groups for the purpose of better service. There is a segment of customer who is the big spender but what if they purchased only once or how recently they purchased? Do they often purchase our product? Also, It helps managers to run an effective promotional campaign for personalized service.<br>

- Recency (R): Who have purchased recently? Number of days since last purchase (least recency)
- Frequency (F): Who has purchased frequently? It means the total number of purchases. ( high frequency)
- Monetary Value(M): Who have high purchase amount? It means the total money customer spent (high monetary value)

In [15]:
## Let's get the maximum date 
max_date = marge_trans_cust['transaction_date'].max()
max_date

Timestamp('2017-12-30 00:00:00')

In [16]:
# Make FRM Tables
rfm_table = marge_trans_cust.groupby(['customer_id']).agg({'transaction_date': lambda date : (max_date - date.max()).days,
                                                            'product_id' : lambda prod_id : len(prod_id), 
                                                            'Profit' : lambda p : sum(p)})
rfm_table
# Transcation date= Last day se kitne din pehle transcation kiya ha vo din diye
# Product_id = total productr mean total transcation in whole duration
# Pofit = Total profit gain by every customer in whole duration

,transaction_date,product_id,Profit
customer_id,,,
1,7,11,3018.09
2,128,3,2226.26
3,102,8,3362.81
4,195,2,220.57
5,16,6,2394.94
...,...,...,...
3496,256,4,2045.84
3497,52,3,1648.32
3498,127,6,3147.33


In [17]:
## CHnage name like (recency, frequency, monetary)

rfm_table.rename(columns = {'transaction_date': 'Recency',
                           'product_id': 'Frequency',
                           'Profit': 'Monetary'}, inplace = True)
rfm_table.columns

Index(['Recency', 'Frequency', 'Monetary'], dtype='object')

In [22]:
# Dividing the recency, frequency and monetary into 4 quartiles (min, 25%, 50%, 75% and max).
# These values will help us to calculate RFM score for a customer and classify based on their RFM score.

# let's recency convert into 4 part [small, midean, big, biggest]
rfm_table['r_quartile'] = pd.qcut(rfm_table['Recency'],4,['4','3','2','1']) 

#let's frequency convert into 4 part [small, midean, big, biggest]
rfm_table['f_quartile'] = pd.qcut(rfm_table['Frequency'],4,['1','2','3','4']) 


rfm_table['m_quartile'] = pd.qcut(rfm_table['Monetary'],4,['1','2','3','4']) 



In [23]:
rfm_table

,Recency,Frequency,Monetary,r_quartile,f_quartile,m_quartile
customer_id,,,,,,
1,7,11,3018.09,4,4,3
2,128,3,2226.26,1,1,2
3,102,8,3362.81,1,4,3
4,195,2,220.57,1,1,1
5,16,6,2394.94,4,2,2
...,...,...,...,...,...,...
3496,256,4,2045.84,1,1,2
3497,52,3,1648.32,2,1,1
3498,127,6,3147.33,1,2,3


In [25]:
# Let's Calculate RFM Score = 100* r_quartile + 10* f_quartile + m_quartile

rfm_table['RFM_Score'] = 100 * rfm_table['r_quartile'].astype(int) + 10 * rfm_table['f_quartile'].astype(int) + rfm_table['m_quartile'].astype(int)

,Recency,Frequency,Monetary,r_quartile,f_quartile,m_quartile,RFM_Score
customer_id,,,,,,,
1,7,11,3018.09,4,4,3,443
2,128,3,2226.26,1,1,2,112
3,102,8,3362.81,1,4,3,143
4,195,2,220.57,1,1,1,111
5,16,6,2394.94,4,2,2,422
...,...,...,...,...,...,...,...
3496,256,4,2045.84,1,1,2,112
3497,52,3,1648.32,2,1,1,211
3498,127,6,3147.33,1,2,3,123


In [26]:
# Assigning a title to a cuustomer.
# Platinum corresponds to highest range of RFM score down to Bronze to lowest range of RFM score

rfm_table['Customer_title'] = pd.qcut(rfm_table['RFM_Score'], 4, ['Bronze','Silver','Gold','Platinum'])
rfm_table.head()

,Recency,Frequency,Monetary,r_quartile,f_quartile,m_quartile,RFM_Score,Customer_title
customer_id,,,,,,,,
1,7,11,3018.09,4,4,3,443,Platinum
2,128,3,2226.26,1,1,2,112,Bronze
3,102,8,3362.81,1,4,3,143,Bronze
4,195,2,220.57,1,1,1,111,Bronze
5,16,6,2394.94,4,2,2,422,Platinum


# Merging both RFM Table with Transaction and Customer Tables

    The RFM_Table dataframe is merged with the Transactions and Customer Demographics datasets,
    to gain depper insights of Customer Segemnts along with transactions. 
    The dataframes are joined based on customer_ids from both the datasets

In [27]:
cust_trans_rfm = pd.merge(marge_trans_cust, rfm_table, left_on = 'customer_id', right_on = 'customer_id', how = 'inner' )
cust_trans_rfm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19354 entries, 0 to 19353
Data columns (total 34 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   transaction_id                       19354 non-null  int64         
 1   product_id                           19354 non-null  int64         
 2   customer_id                          19354 non-null  int64         
 3   transaction_date                     19354 non-null  datetime64[ns]
 4   online_order                         19354 non-null  float64       
 5   order_status                         19354 non-null  object        
 6   brand                                19354 non-null  object        
 7   product_line                         19354 non-null  object        
 8   product_class                        19354 non-null  object        
 9   product_size                         19354 non-null  object        
 10  list_price

##### Data type look fine grate.

In [32]:
#### Let's Create a columns of Age group

def age_group(age):
    if age < 30:
        return '20-30'
    elif age < 40:
        return '30-40'
    elif age <50:
        return '40-50'
    elif age < 60:
        return '50-60'
    elif age < 70:
        return '60-70'
    elif age < 80:
        return '70-80'
    elif age < 90:
        return '80-90'
    else:
        return '90-100'
    
cust_trans_rfm['Age_group'] = cust_trans_rfm['Age'].apply(age_group)
    

#### Creating a Detail Customer title / tag based on RFM Score
    An extended version of customer title is made which divides the entire number of customers 
    into 11 groups. The groups are mainly Platinum Customers, Very Loyal, Becoming Loyal, 
    Recent Customers, Potential Customers, Late Bloomer, Loosing Customers, 
    High Risk Customers, Almost Lost Customers, Evasive Customers and Lost Customers.
    <br> The demarkation of customers into the above mentioned groups is based on their RFM scores.

In [48]:
# Function as a lookup to appropiate customer titles based on RFM score.

def cust_score_title(rfm_score):
    
    
    if rfm_score >= 444:
        return 'Platinum Customer'
    elif rfm_score >=433 and rfm_score < 444:
        return 'Very Loyal'
    elif rfm_score >=421 and rfm_score < 433:
        return 'Becoming Loyal'
    elif rfm_score >=344 and rfm_score < 421:
        return 'Recent Customer'
    elif rfm_score >=323 and rfm_score < 344:
        return 'Potential Customer'
    elif rfm_score >=311 and rfm_score < 323:
        return 'Late Bloomer'
    elif rfm_score >=224 and rfm_score < 311:
        return 'Loosing Customer'
    elif rfm_score >=212 and rfm_score < 224:
        return 'High Risk Customer'
    elif rfm_score >=124 and rfm_score < 212:
        return 'Almost Lost Customer'
    elif rfm_score >=112 and rfm_score < 124:
        return 'Evasive Customer'
    else :
        return 'Lost Customer'
    
cust_trans_rfm['detail_cust_title'] = cust_trans_rfm['RFM_Score'].apply(cust_score_title)

In [49]:
# Function to provide ranks to the customers based on their title.

def get_rank(title):
    
    if title=='Platinum Customer':
        return 1
    elif title=='Very Loyal':
        return 2
    elif title == 'Becoming Loyal':
        return 3
    elif title == 'Recent Customer':
        return 4
    elif title=='Potential Customer':
        return 5
    elif title == 'Late Bloomer':
        return 6
    elif title == 'Loosing Customer':
        return 7
    elif title=='High Risk Customer':
        return 8
    elif title == 'Almost Lost Customer':
        return 9
    elif title == 'Evasive Customer':
        return 10
    else :
        return 11

In [52]:
cust_trans_rfm['rank']=cust_trans_rfm['detail_cust_title'].apply(get_rank)

In [53]:
cust_trans_rfm

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,...,Frequency,Monetary,r_quartile,f_quartile,m_quartile,RFM_Score,Customer_title,Age_group,detail_cust_title,rank
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,...,3,645.99,2,1,1,211,Bronze,60-70,Almost Lost Customer,9
1,11065,1,2950,2017-10-16,0.0,Approved,Giant Bicycles,Standard,medium,medium,...,3,645.99,2,1,1,211,Bronze,60-70,Almost Lost Customer,9
2,18923,62,2950,2017-04-26,0.0,Approved,Solex,Standard,medium,medium,...,3,645.99,2,1,1,211,Bronze,60-70,Almost Lost Customer,9
3,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,...,7,4179.11,3,3,4,334,Gold,40-50,Potential Customer,5
4,6862,4,3120,2017-10-05,0.0,Approved,Giant Bicycles,Standard,high,medium,...,7,4179.11,3,3,4,334,Gold,40-50,Potential Customer,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19349,18735,45,3168,2017-02-21,1.0,Approved,Solex,Standard,medium,medium,...,4,5170.51,1,1,4,114,Bronze,40-50,Evasive Customer,10
19350,18789,83,3168,2017-04-11,0.0,Approved,Solex,Touring,medium,large,...,4,5170.51,1,1,4,114,Bronze,40-50,Evasive Customer,10
19351,19113,3,3168,2017-06-03,0.0,Approved,Trek Bicycles,Standard,medium,large,...,4,5170.51,1,1,4,114,Bronze,40-50,Evasive Customer,10
19352,19912,3,3168,2017-09-30,0.0,Approved,Trek Bicycles,Standard,medium,large,...,4,5170.51,1,1,4,114,Bronze,40-50,Evasive Customer,10


In [56]:
print('Finaaly cleaned, now totla number of rows = ', cust_trans_rfm.shape[0])
print('Finaaly cleaned, now totla number of columns = ', cust_trans_rfm.shape[1])

Finaaly cleaned, now totla number of rows =  19354
Finaaly cleaned, now totla number of columns =  37


# Exporting to CSV File

In [57]:
cust_trans_rfm.to_csv('Customer_Trans_RFM_Analysis.csv', index=False)